In [ ]:
import numpy as np
import torch
import tntorch as tn
from scipy.stats import multivariate_normal

def make_psd(cov):
    # 1. Simmetrizza
    cov_sym = 0.5 * (cov + cov.T)
    
    # 2. Autovalori e autovettori
    eigvals, eigvecs = np.linalg.eigh(cov_sym)

    # 3. Azzeramento degli autovalori negativi
    eigvals_clipped = np.clip(eigvals, a_min=1e-6, a_max=None)  # imposta min=1e-6 per evitare problemi numerici

    # 4. Ricostruzione matrice PSD
    cov_psd = eigvecs @ np.diag(eigvals_clipped) @ eigvecs.T
    return cov_psd



# Parametri Gaussiana 2D
mu = np.array([0.10, 0.10])
cov_matrix = make_psd(np.array([[0.20, 0.25], [0.25, 0.07]]))  # Matrice simmetrica PSD

# Discretizzazione: 5 qubit per dimensione → 2^5 = 32 punti
num_qubits_per_dim = 5
grid_size = 2 ** num_qubits_per_dim

# Griglia per x e y
x_grid = np.linspace(mu[0] - 3*np.sqrt(cov_matrix[0,0]), mu[0] + 3*np.sqrt(cov_matrix[0,0]), grid_size)
y_grid = np.linspace(mu[1] - 3*np.sqrt(cov_matrix[1,1]), mu[1] + 3*np.sqrt(cov_matrix[1,1]), grid_size)

# Valori della PDF
dist = multivariate_normal(mean=mu, cov=cov_matrix)
pdf_values = np.zeros((grid_size, grid_size))
for i, x in enumerate(x_grid):
    for j, y in enumerate(y_grid):
        pdf_values[i,j] = dist.pdf([x, y])

# Converti in tensore e applica TT-cross con 10 core
tensor = torch.tensor(pdf_values, dtype=torch.float32)
tt = tn.Tensor(tensor, ranks_tt=8)  # Rank massimo 8

# Forza la struttura a 10 core (1 core/qubit)
tt_reshaped = tt.reshape([2] * 10)  # 10 dimensioni binarie (2 stati/qubit)
print(tt_reshaped)